#### List S3 Buckets and Create an S3 Bucket

In [11]:
import boto3

s3 = boto3.client('s3')

bucket_name = 'aduniquebucket'

In [12]:
response = s3.list_buckets()

In [ ]:
def create_bucket(bucket_name):
    s3.create_bucket(Bucket=bucket_name)
    print("Bucket is created")


# S3 bucket names must be globally unique,
# Bucket names must be 3 to 63 characters and unique within the global namespace. 
# Bucket names must also begin and end with a letter or number. Valid characters are a-z, 0-9, periods (.), and hyphens (-)
create_bucket(bucket_name)

Bucket is created


## Upload a Single File to S3

In [14]:
import os

def upload_file(file_path, object_name=None):
    if object_name is None:
        object_name = os.path.basename(file_path)

    s3.upload_file(file_path, bucket_name, object_name)


In [15]:
upload_file('data/index.html')

In [16]:
upload_file('data/index.html', 'somefile.html') #rename the file in the bucket here somewhat is renamed to index.html

- list all objects in a bucket

In [17]:
def list_objects():
    response = s3.list_objects_v2(Bucket=bucket_name)
    for obj in response['Contents']:
        print(obj['Key'])

list_objects()

index.html
somefile.html


##### Download S3 File to Local System

In [18]:
def download_file(object_name, file_path):
    if not os.path.exists(os.path.dirname(file_path)):
        os.makedirs(os.path.dirname(file_path))

    s3.download_file(bucket_name, object_name, file_path)


download_file('index.html', 'data_download/index.html')

##### Upload a Folder to S3

In [19]:
def upload_directory(directory_path, s3_prefix):
    for root, dirs, files in os.walk('data'):
        for file in files:
            file_path = os.path.join(root, file).replace("\\", "/")
            relpath = os.path.relpath(file_path, directory_path)
            s3_key = os.path.join(s3_prefix, relpath).replace("\\", "/")
            
            s3.upload_file(file_path, bucket_name, s3_key)

upload_directory('data', 's3_data')

##### Download S3 Folder to Local System

In [20]:
local_path = 's3_download_1'
s3_prefix = 's3_data'

def download_dir(local_path, s3_prefix):
    os.makedirs(local_path, exist_ok=True)
    paginator = s3.get_paginator('list_objects_v2')
    for result in paginator.paginate(Bucket=bucket_name, Prefix=s3_prefix):
        if 'Contents' in result:
            for key in result['Contents']:
                s3_key = key['Key']

                local_file = os.path.join(local_path, os.path.relpath(s3_key, s3_prefix))
                # os.makedirs(os.path.dirname(local_file), exist_ok=True)

                s3.download_file(bucket_name, s3_key, local_file)



download_dir(local_path, s3_prefix)

##### Delete All Files in an S3 Bucket

In [21]:
def delete_objects():
    response = s3.list_objects_v2(Bucket=bucket_name)

    if 'Contents' in response:

        for obj in response['Contents']:
            s3.delete_object(Bucket=bucket_name, Key=obj['Key'])



delete_objects()

In [22]:
s3.delete_bucket(Bucket=bucket_name)

{'ResponseMetadata': {'RequestId': 'H5D34SMXYG3ES5VD',
  'HostId': 'Gg6cPFkD0EGoloLkqXmpcjQvX36rF4h+A2AmITIaJo1jQORzUZlLoEzC8x57fTmjUOxh8Cnt2saoUKYdT1oN3oZTCY4wAM/aW8JrKfY4LkQ=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'Gg6cPFkD0EGoloLkqXmpcjQvX36rF4h+A2AmITIaJo1jQORzUZlLoEzC8x57fTmjUOxh8Cnt2saoUKYdT1oN3oZTCY4wAM/aW8JrKfY4LkQ=',
   'x-amz-request-id': 'H5D34SMXYG3ES5VD',
   'date': 'Wed, 02 Jul 2025 05:58:47 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}